In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=89ee318cef72b2631fd979499f0f4673cd93de53718b7a688dc9c46f4a84cf7a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [243]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, countDistinct, desc, count, udf, array, explode, lit ,sum, regexp_replace, trim, avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType, ArrayType
import pandas as pd

## **Table Athlete - Global Investigation/Traitement**

In [104]:
spark = SparkSession.builder \
    .appName("MultiFormatDataLoading") \
    .config("spark.jars.packages",
            "com.databricks:spark-xml_2.12:0.14.0,"
            "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

json_schema = StructType([
    StructField("athlete_url", StringType(), True),
    StructField("athlete_full_name", StringType(), True),
    StructField("games_participations", IntegerType(), True),
    StructField("first_game", StringType(), True),
    StructField("athlete_year_birth", FloatType(), True),
    StructField("athlete_medals", StringType(), True),
    StructField("bio", StringType(), True)
])

df_athlete = spark.read.json('/content/olympic_athletes.json')

In [106]:
df_athlete.printSchema()

root
 |-- athlete_full_name: string (nullable = true)
 |-- athlete_medals: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_year_birth: double (nullable = true)
 |-- bio: string (nullable = true)
 |-- first_game: string (nullable = true)
 |-- games_participations: long (nullable = true)



In [107]:
df_athlete.show(truncate=False)

+----------------------+------------------------------+-------------------------------------------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [108]:
# Cast Float to Int - Year of birth
df_athlete = df_athlete.withColumn("athlete_year_birth", col("athlete_year_birth").cast("int"))
df_athlete.show(truncate=False)

+----------------------+------------------------------+-------------------------------------------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Séparer en trois colonnes disctinctes les médailles par athlete

In [109]:
distinct_medals = df_athlete.select("athlete_medals").distinct()
distinct_medals.show(50,truncate=False)

+---------------------------------------------+
|athlete_medals                               |
+---------------------------------------------+
|\n\n\n4\n\nG\n\n\n\n1\n\nS\n\n\n\n2\n\nB\n\n |
|\n\n\n9\n\nG\n\n\n\n3\n\nS\n\n               |
|\n\n\n6\n\nG\n\n\n\n2\n\nS\n\n\n\n2\n\nB\n\n |
|\n\n\n5\n\nG\n\n\n\n1\n\nS\n\n               |
|\n\n\n4\n\nG\n\n\n\n3\n\nS\n\n               |
|\n\n\n2\n\nG\n\n\n\n3\n\nB\n\n               |
|\n\n\n5\n\nG\n\n\n\n3\n\nS\n\n\n\n1\n\nB\n\n |
|\n\n\n1\n\nG\n\n\n\n3\n\nS\n\n\n\n3\n\nB\n\n |
|\n\n\n3\n\nS\n\n\n\n2\n\nB\n\n               |
|\n\n\n4\n\nS\n\n\n\n2\n\nB\n\n               |
|\n\n\n9\n\nG\n\n\n\n1\n\nS\n\n               |
|\n\n\n1\n\nS\n\n                             |
|\n\n\n1\n\nS\n\n\n\n3\n\nB\n\n               |
|\n\n\n4\n\nG\n\n                             |
|\n\n\n23\n\nG\n\n\n\n3\n\nS\n\n\n\n2\n\nB\n\n|
|\n\n\n2\n\nG\n\n\n\n4\n\nS\n\n\n\n1\n\nB\n\n |
|\n\n\n2\n\nG\n\n\n\n3\n\nS\n\n\n\n3\n\nB\n\n |
|\n\n\n1\n\nG\n\n\n\n4\n\nS\n\n\n\n2\n\n

In [110]:
#Remove "\n"
df_athlete = df_athlete.withColumn("athlete_medals", regexp_replace(col("athlete_medals"), "\n", " "))

#Remove " "
df_athlete = df_athlete.withColumn("athlete_medals", trim(col("athlete_medals")))

distinct_medals = df_athlete.select("athlete_medals").distinct()
distinct_medals.show(truncate=False)

+--------------------+
|athlete_medals      |
+--------------------+
|3  G    2  S        |
|4  G    2  S    2  B|
|3  G    6  S        |
|3  G    4  S    2  B|
|1  G    1  B        |
|2  S    4  B        |
|4  G    1  B        |
|4  G    1  S    1  B|
|1  G    5  S    3  B|
|4  G    2  B        |
|3  G    5  S        |
|4  G    2  S    3  B|
|1  G    5  S    1  B|
|2  G    2  B        |
|3  G    3  B        |
|5  G    2  S    4  B|
|5  G    1  S        |
|1  G    1  S    3  B|
|2  G    2  S    4  B|
|1  G    4  S        |
+--------------------+
only showing top 20 rows



In [111]:
#Split Colonne pour chaque type de médaille

gold_pattern = r"(\d+)\s*G"
silver_pattern = r"(\d+)\s*S"
bronze_pattern = r"(\d+)\s*B"

df_athlete = df_athlete.withColumn("total_gold", regexp_extract(col("athlete_medals"), gold_pattern, 1).cast("int"))
df_athlete = df_athlete.withColumn("total_silver", regexp_extract(col("athlete_medals"), silver_pattern, 1).cast("int"))
df_athlete = df_athlete.withColumn("total_bronze", regexp_extract(col("athlete_medals"), bronze_pattern, 1).cast("int"))

# Remplacer Null par 0
df_athlete = df_athlete.fillna({'athlete_medals': '0', 'total_gold': 0, 'total_silver': 0, 'total_bronze': 0})

#Somme des colonnes
df_athlete = df_athlete.withColumn("total_medals", col("total_gold") + col("total_silver") + col("total_bronze"))

df_athlete.select("athlete_medals", "total_gold", "total_silver", "total_bronze", "total_medals").show(truncate=False)

+--------------+----------+------------+------------+------------+
|athlete_medals|total_gold|total_silver|total_bronze|total_medals|
+--------------+----------+------------+------------+------------+
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|1  S          |0         |1           |0           |1           |
|1  B          |0         |0           |1           |1           |
|1  B          |0         |0           |1           |1           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0           |
|0             |0         |0           |0           |0        

In [112]:
#Ajout colonne "is_new" pour savoir si un athlète n'a fait qu'une participation
df_athlete = df_athlete.withColumn("is_new", when(col("games_participations") == 1, 1).otherwise(0))
df_athlete.show()

+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+
|   athlete_full_name|athlete_medals|         athlete_url|athlete_year_birth|                 bio|      first_game|games_participations|total_gold|total_silver|total_bronze|total_medals|is_new|
+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+
|Cooper WOODS-TOPA...|             0|https://olympics....|              2000|                NULL|    Beijing 2022|                   1|         0|           0|           0|           0|     1|
|      Felix ELOFSSON|             0|https://olympics....|              1995|                NULL|PyeongChang 2018|                   2|         0|           0|           0|           0|     0|
|       Dylan WALCZYK|        

In [113]:
#Remove "\n"
df_athlete = df_athlete.withColumn("bio", regexp_replace(col("bio"), "\n", " "))

#Remove " "
df_athlete = df_athlete.withColumn("bio", trim(col("bio")))

df_athlete.show()

+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+
|   athlete_full_name|athlete_medals|         athlete_url|athlete_year_birth|                 bio|      first_game|games_participations|total_gold|total_silver|total_bronze|total_medals|is_new|
+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+
|Cooper WOODS-TOPA...|             0|https://olympics....|              2000|                NULL|    Beijing 2022|                   1|         0|           0|           0|           0|     1|
|      Felix ELOFSSON|             0|https://olympics....|              1995|                NULL|PyeongChang 2018|                   2|         0|           0|           0|           0|     0|
|       Dylan WALCZYK|        

In [114]:
average_year_birth = df_athlete.select(avg(col("athlete_year_birth"))).collect()[0][0]
df_athlete = df_athlete.fillna({'athlete_year_birth': average_year_birth})

current_year = 2024

# Création colonne (athlete_age)
df_athlete = df_athlete.withColumn("athlete_age", (lit(current_year) - col("athlete_year_birth")).cast("int"))
df_athlete.select("athlete_full_name", "athlete_year_birth", "athlete_age").show(truncate=False)

+----------------------+------------------+-----------+
|athlete_full_name     |athlete_year_birth|athlete_age|
+----------------------+------------------+-----------+
|Cooper WOODS-TOPALOVIC|2000              |24         |
|Felix ELOFSSON        |1995              |29         |
|Dylan WALCZYK         |1993              |31         |
|Olli PENTTALA         |1995              |29         |
|Dmitriy REIKHERD      |1989              |35         |
|Matt GRAHAM           |1994              |30         |
|Ikuma HORISHIMA       |1997              |27         |
|Daichi HARA           |1997              |27         |
|Laurent DUMAIS        |1996              |28         |
|James MATHESON        |1995              |29         |
|Pavel KOLMAKOV        |1996              |28         |
|Kosuke SUGIMOTO       |1994              |30         |
|Brodie SUMMERS        |1993              |31         |
|Severi VIERELA        |2001              |23         |
|Marco TADE            |1995              |29   

In [115]:
df_athlete = df_athlete.fillna({'bio': ''})
df_athlete.show()

+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+-----------+
|   athlete_full_name|athlete_medals|         athlete_url|athlete_year_birth|                 bio|      first_game|games_participations|total_gold|total_silver|total_bronze|total_medals|is_new|athlete_age|
+--------------------+--------------+--------------------+------------------+--------------------+----------------+--------------------+----------+------------+------------+------------+------+-----------+
|Cooper WOODS-TOPA...|             0|https://olympics....|              2000|                    |    Beijing 2022|                   1|         0|           0|           0|           0|     1|         24|
|      Felix ELOFSSON|             0|https://olympics....|              1995|                    |PyeongChang 2018|                   2|         0|           0|           0|   

In [116]:
# Connection settings
url = "jdbc:postgresql://postgresql-hackathon2024.alwaysdata.net:5432/hackathon2024_2"
properties = {
    "user": "hackathon2024",
    "password": "bsaebno2001",
    "driver": "org.postgresql.Driver"
}

df_athlete.write.jdbc(url=url, table="olympic_athletes", mode="append", properties=properties)

In [136]:
spark.stop()

## **Table Result - Global Investigation/Traitement**

In [172]:
spark = SparkSession.builder \
    .appName("Html_loading") \
    .getOrCreate()

In [173]:
html_df = pd.read_html('/content/olympic_results.html')[0]

html_df.to_csv('/content/olympic_results.html', index=False)
df_result = spark.read.option("header", "true").option("inferSchema", "true").csv('/content/olympic_results.html')

In [174]:
df_result.printSchema()

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- discipline_title: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- athletes: string (nullable = true)
 |-- rank_equal: string (nullable = true)
 |-- rank_position: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- value_unit: string (nullable = true)
 |-- value_type: string (nullable = true)



In [175]:
# Retirer les épreuves mixtes
df_result = df_result.filter(col("participant_type") == "Athlete")
df_result.show()

+----------+----------------+------------+------------+----------------+----------+--------+----------+-------------+--------------------+------------+---------------------+--------------------+--------------------+----------+----------+
|Unnamed: 0|discipline_title| event_title|   slug_game|participant_type|medal_type|athletes|rank_equal|rank_position|        country_name|country_code|country_3_letter_code|         athlete_url|   athlete_full_name|value_unit|value_type|
+----------+----------------+------------+------------+----------------+----------+--------+----------+-------------+--------------------+------------+---------------------+--------------------+--------------------+----------+----------+
|        30|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NULL|    NULL|     False|            6|           Australia|          AU|                  AUS|https://olympics....|Cooper WOODS-TOPA...|     76.74|    POINTS|
|        31|Freestyle Skiing|Men's Moguls|beijin

In [176]:
df_result = df_result.drop(df_result.columns[0])

In [177]:
df_result = df_result.drop("value_unit", "value_type", "index", "country_code", "athletes", "rank_equal")
df_result.show()

+----------------+------------+------------+----------------+----------+-------------+--------------------+---------------------+--------------------+--------------------+
|discipline_title| event_title|   slug_game|participant_type|medal_type|rank_position|        country_name|country_3_letter_code|         athlete_url|   athlete_full_name|
+----------------+------------+------------+----------------+----------+-------------+--------------------+---------------------+--------------------+--------------------+
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NULL|            6|           Australia|                  AUS|https://olympics....|Cooper WOODS-TOPA...|
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NULL|           17|              Sweden|                  SWE|https://olympics....|      Felix ELOFSSON|
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NULL|           16|United States of ...|                  USA|https://oly

In [178]:
df_result = df_result.fillna({'medal_type': 'NONE'})
df_result.show()

+----------------+------------+------------+----------------+----------+-------------+--------------------+---------------------+--------------------+--------------------+
|discipline_title| event_title|   slug_game|participant_type|medal_type|rank_position|        country_name|country_3_letter_code|         athlete_url|   athlete_full_name|
+----------------+------------+------------+----------------+----------+-------------+--------------------+---------------------+--------------------+--------------------+
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NONE|            6|           Australia|                  AUS|https://olympics....|Cooper WOODS-TOPA...|
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NONE|           17|              Sweden|                  SWE|https://olympics....|      Felix ELOFSSON|
|Freestyle Skiing|Men's Moguls|beijing-2022|         Athlete|      NONE|           16|United States of ...|                  USA|https://oly

In [179]:
columns = df_result.columns
null_counts = df_result.select([sum(col(column).isNull().cast("int")).alias(column) for column in columns])
null_counts.show()

+----------------+-----------+---------+----------------+----------+-------------+------------+---------------------+-----------+-----------------+
|discipline_title|event_title|slug_game|participant_type|medal_type|rank_position|country_name|country_3_letter_code|athlete_url|athlete_full_name|
+----------------+-----------+---------+----------------+----------+-------------+------------+---------------------+-----------+-----------------+
|               0|          0|        0|               0|         0|          163|           1|                    1|        453|                1|
+----------------+-----------+---------+----------------+----------+-------------+------------+---------------------+-----------+-----------------+



In [180]:
df_result = df_result.dropna(subset=["athlete_url"])
df_result.show(truncate=False)

+----------------+------------+------------+----------------+----------+-------------+------------------------+---------------------+-------------------------------------------------------+----------------------+
|discipline_title|event_title |slug_game   |participant_type|medal_type|rank_position|country_name            |country_3_letter_code|athlete_url                                            |athlete_full_name     |
+----------------+------------+------------+----------------+----------+-------------+------------------------+---------------------+-------------------------------------------------------+----------------------+
|Freestyle Skiing|Men's Moguls|beijing-2022|Athlete         |NONE      |6            |Australia               |AUS                  |https://olympics.com/en/athletes/cooper-woods-topalovic|Cooper WOODS-TOPALOVIC|
|Freestyle Skiing|Men's Moguls|beijing-2022|Athlete         |NONE      |17           |Sweden                  |SWE                  |https://olympic

In [181]:
# Connection settings
url = "jdbc:postgresql://postgresql-hackathon2024.alwaysdata.net:5432/hackathon2024_2"
properties = {
    "user": "hackathon2024",
    "password": "bsaebno2001",
    "driver": "org.postgresql.Driver"
}

df_result.write.jdbc(url=url, table="olympic_results", mode="append", properties=properties)

In [182]:
spark.stop()


## **Table Medals - Global Investigation/Traitement**

In [186]:
spark = SparkSession.builder \
    .appName("Excel_Loading") \
    .getOrCreate()

In [187]:
df_medal_pd = pd.read_excel('/content/olympic_medals.xlsx', engine='openpyxl')

df_medal = spark.createDataFrame(df_medal_pd)
df_medal.show(5)

+----------+----------------+------------+-------------+------------+----------+----------------+-----------------+--------------------+--------------------+------------+------------+---------------------+
|Unnamed: 0|discipline_title|   slug_game|  event_title|event_gender|medal_type|participant_type|participant_title|         athlete_url|   athlete_full_name|country_name|country_code|country_3_letter_code|
+----------+----------------+------------+-------------+------------+----------+----------------+-----------------+--------------------+--------------------+------------+------------+---------------------+
|         0|         Curling|beijing-2022|Mixed Doubles|       Mixed|      GOLD|        GameTeam|            Italy|https://olympics....|Stefania CONSTANTINI|       Italy|          IT|                  ITA|
|         1|         Curling|beijing-2022|Mixed Doubles|       Mixed|      GOLD|        GameTeam|            Italy|https://olympics....|        Amos MOSANER|       Italy|      

In [188]:
df_medal.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- event_gender: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- participant_title: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)



In [191]:
df_medal = df_medal.drop(df_medal.columns[0])

In [192]:
df_medal = df_medal.dropna(subset=["athlete_url"])

In [193]:
df_medal.select("participant_type").distinct().show(truncate=False)

+----------------+
|participant_type|
+----------------+
|Athlete         |
|GameTeam        |
+----------------+



In [194]:
# Retirer les épreuves mixtes
df_medal = df_medal.filter(col("participant_type") == "Athlete")
df_medal.show()

+------------+--------------------+------------+----------+----------------+-----------------+--------------------+-------------------+--------------------+------------+---------------------+
|   slug_game|         event_title|event_gender|medal_type|participant_type|participant_title|         athlete_url|  athlete_full_name|        country_name|country_code|country_3_letter_code|
+------------+--------------------+------------+----------+----------------+-----------------+--------------------+-------------------+--------------------+------------+---------------------+
|beijing-2022|        Men's Moguls|         Men|    SILVER|         Athlete|              NaN|https://olympics....|   Mikael KINGSBURY|              Canada|          CA|                  CAN|
|beijing-2022|        Men's Moguls|         Men|      GOLD|         Athlete|              NaN|https://olympics....|    Walter WALLBERG|              Sweden|          SE|                  SWE|
|beijing-2022|        Men's Moguls|     

In [195]:
df_medal.select("participant_title").distinct().show(truncate=False)

+-----------------+
|participant_title|
+-----------------+
|NaN              |
+-----------------+



In [ ]:
df_medal = df_medal.drop(df_medal.participant_title,df_medal.country_code)

In [200]:
df_medal.show()

+------------+--------------------+------------+----------+----------------+--------------------+-------------------+--------------------+---------------------+
|   slug_game|         event_title|event_gender|medal_type|participant_type|         athlete_url|  athlete_full_name|        country_name|country_3_letter_code|
+------------+--------------------+------------+----------+----------------+--------------------+-------------------+--------------------+---------------------+
|beijing-2022|        Men's Moguls|         Men|    SILVER|         Athlete|https://olympics....|   Mikael KINGSBURY|              Canada|                  CAN|
|beijing-2022|        Men's Moguls|         Men|      GOLD|         Athlete|https://olympics....|    Walter WALLBERG|              Sweden|                  SWE|
|beijing-2022|        Men's Moguls|         Men|    BRONZE|         Athlete|https://olympics....|    Ikuma HORISHIMA|               Japan|                  JPN|
|beijing-2022|Men's Freeski Hal...

In [201]:
# Connection settings
url = "jdbc:postgresql://postgresql-hackathon2024.alwaysdata.net:5432/hackathon2024_2"
properties = {
    "user": "hackathon2024",
    "password": "bsaebno2001",
    "driver": "org.postgresql.Driver"
}

df_medal.write.jdbc(url=url, table="olympic_medals", mode="append", properties=properties)

In [202]:
spark.stop()

## **Table Hosts - Global Investigation/Traiment**

In [235]:
import xml.etree.ElementTree as ET

tree = ET.parse('/content/olympic_hosts.xml')
root = tree.getroot()

data = []
for row in root.findall('row'):
    row_data = {}
    for child in row:
        row_data[child.tag] = child.text
    data.append(row_data)

# Convertir la liste de dictionnaires en DataFrame Pandas
df_pandas = pd.DataFrame(data)
print(df_pandas.head())


  index         game_slug         game_end_date       game_start_date  \
0     0      beijing-2022  2022-02-20T12:00:00Z  2022-02-04T15:00:00Z   
1     1        tokyo-2020  2021-08-08T14:00:00Z  2021-07-23T11:00:00Z   
2     2  pyeongchang-2018  2018-02-25T08:00:00Z  2018-02-08T23:00:00Z   
3     3          rio-2016  2016-08-21T21:00:00Z  2016-08-05T12:00:00Z   
4     4        sochi-2014  2014-02-23T16:00:00Z  2014-02-07T04:00:00Z   

        game_location         game_name game_season game_year  
0               China      Beijing 2022      Winter      2022  
1               Japan        Tokyo 2020      Summer      2020  
2   Republic of Korea  PyeongChang 2018      Winter      2018  
3              Brazil          Rio 2016      Summer      2016  
4  Russian Federation        Sochi 2014      Winter      2014  


In [236]:
spark = SparkSession.builder \
    .appName("XML_Loading") \
    .getOrCreate()

In [237]:
df_host = spark.createDataFrame(df_pandas)

In [238]:
df_host = df_host.drop(df_host.columns[0], df_host.game_end_date, df_host.game_start_date)

In [239]:
# total valeurs nulles dans la colonne game_slug
null_count_spark = df_host.select(sum(col('game_slug').isNull().cast("int")).alias('null_count')).collect()[0]['null_count']
print(null_count_spark)


0


In [241]:
# Connection settings
url = "jdbc:postgresql://postgresql-hackathon2024.alwaysdata.net:5432/hackathon2024_2"
properties = {
    "user": "hackathon2024",
    "password": "bsaebno2001",
    "driver": "org.postgresql.Driver"
}

df_host.write.jdbc(url=url, table="olympic_hosts", mode="append", properties=properties)

In [242]:
spark.stop()